# Generating Vegan Recipes

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/src/examples/recipe_generation.ipynb).

In this example, we will use Guardrails to generate vegan mac and cheese recipe.

## Objective

We want to generate a vegan Mac-n-Cheese recipe as a list of ingredients and instructions. We will use Guardrails to make sure the recipe is vegan.

In [1]:
import guardrails as gd
from rich import print

/Users/dtam/.pyenv/versions/3.12.3/envs/litellm/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](/docs/how_to_guides/rail).  We will also show the same RAIL spec in a code-first format using a Pydantic model.

First we define a custom Validator:

In [2]:
from guardrails.validators import (
    Validator,
    register_validator,
    ValidationResult,
    PassResult,
    FailResult,
)
from typing import Dict, Any

NON_VEGAN_INGREDIENTS = ["butter", "milk", "eggs", "cheese", "cream", "yogurt"]
SUBSTITUTIONS = {
    "butter": "margarine",
    "milk": "soy milk",
    "eggs": "flax eggs",
    "cheese": "vegan cheese",
    "cream": "soy cream",
    "yogurt": "soy yogurt",
}


@register_validator(name="is-vegan", data_type="string")
class IsVegan(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        global NON_VEGAN_INGREDIENTS, SUBSTITUTIONS

        # Make sure the ingredient is not in the list of non-vegan ingredients.
        if value.lower() in NON_VEGAN_INGREDIENTS:
            return FailResult(
                error_message=f"Value ${value} is not vegan.",
                # Programmatically fix the value by replacing it with a vegan
                # substitute.
                fix_value=SUBSTITUTIONS[value.lower()],
            )

        return PassResult()

Next we can define our RAIL spec either as a XML string:

In [8]:
rail_str = """
<rail version="0.1">

<output>
    <list name="ingredients" description="What are the ingredients for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="is-vegan" on-fail-is-vegan="fix" />
            <string name="brand" description="Suggested brand for the ingredient (if any)" />
            <bool name="optional" description="Is the ingredient necessary?" />
            <float name="quantity" format="units-imperial" />
            <string name="units" format="units-imperial" />
        </object>
    </list>
    <list name="instructions" description="What are the instructions for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="step" />
        </object>
    </list>
</output>

<messages>
<message role="user">
Generate a recipe for vegan mac and cheese.
${gr.complete_xml_suffix}
</message>
</messages>

</rail>
"""

or a Pydantic model:

In [6]:
from pydantic import BaseModel, Field
from typing import List


prompt = """
Generate a recipe for vegan mac and cheese.
${gr.complete_xml_suffix}
"""


class Ingredient(BaseModel):
    index: int = Field(validators=[("1-indexed", "noop")])
    name: str = Field(validators=[IsVegan(on_fail="fix")])
    brand: str = Field(description="Suggested brand for the ingredient (if any)")
    optional: bool = Field(description="Is the ingredient necessary?")
    quantity: float = Field(
        description="how much of this ingredient to use",
        validators=[("units-imperial", "noop")],
    )
    units: str = Field(validators=[("units-imperial", "noop")])


class Instruction(BaseModel):
    index: int = Field(validators=[("1-indexed", "noop")])
    step: str


class Recipe(BaseModel):
    ingredients: List[Ingredient] = Field(description="What are the ingredients for the recipe?")
    instructions: List[Instruction] = Field(description="What are the instructions for the recipe?")

!!! note
    Here, we create a custom `IsVegan` validator that checks if the ingredient is vegan.
    We also set `on-fail-is-vegan` to `fix`, which in this case means that programatically we will replace the ingredient with a vegan substitute.

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

From the XML string RAIL spec:

In [9]:
guard = gd.Guard.for_rail_string(rail_str)

From the Pydantic model:

In [6]:
guard = gd.Guard.for_pydantic(output_class=Recipe)

/Users/dtam/.pyenv/versions/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/dtam/.pyenv/versions/3.12.1/lib/python3.12/site-packages/guardrails/validator_base.py:152: UserWarning: Validator with id 1-indexed was not found in the registry!  Ignoring...
  warn(f"Validator with id {name} was not found in the registry!  Ignoring...")
/Users/dtam/.pyenv/versions/3.12.1/lib/python3.12/site-packages/guardrails/validator_base.py:152: UserWarning: Validator with id units-imperial was not found in the registry!  Ignoring...
  warn(f"Validator with id {name} was not found in the registry!  Ignoring...")


As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

## Step 3: Wrap the LLM API call with `Guard`

In [10]:
# Add your OPENAI_API_KEY as an environment variable if it's not already set
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

raw_llm_response, validated_response, *rest = guard(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=2048,
    temperature=0,
    model="gpt-4",
)

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


We can see the prompt that was sent to the LLM. The `{document}` param was substituted with the user provided value at runtime.

In [11]:
print(guard.history.first.iterations.first.inputs.messages[0]["content"])

Generate a recipe for vegan mac and cheese.

Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
  <list description="What are the ingredients for the recipe?" name="ingredients" required="true">
    <object required="true">
      <integer format="1-indexed" name="index" required="true"></integer>
      <string format="is-vegan" name="name" required="true"></string>
      <string description="Suggested brand for the ingredient (if any)" name="brand" required="true"></string>
      <bool description="Is the ingredient necessary?" name="optional" required="true"></bool>
      <float format="units-imperial" name="quantity" required="true"></float>
      <string format="units-imperial" name="units" required="true"></string>
    </object>
  </list>
  <list description="What are the instructions for the recipe?" name="instructions" required="true">
    <object required="true">
      <integer format="1-indexed" name="index" required="true"></integer>
      <string name="step" required="true"></string>
    </object>
  </list>
</output>

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [12]:
print(validated_response)

{
    'ingredients': [
        {
            'index': 1,
            'name': 'macaroni',
            'brand': 'Barilla',
            'optional': False,
            'quantity': 8.0,
            'units': 'ounces'
        },
        {
            'index': 2,
            'name': 'raw cashews',
            'brand': "Trader Joe's",
            'optional': False,
            'quantity': 1.0,
            'units': 'cup'
        },
        {
            'index': 3,
            'name': 'nutritional yeast',
            'brand': "Bob's Red Mill",
            'optional': False,
            'quantity': 0.25,
            'units': 'cup'
        },
        {
            'index': 4,
            'name': 'garlic powder',
            'brand': 'McCormick',
            'optional': False,
            'quantity': 1.0,
            'units': 'teaspoon'
        },
        {
            'index': 5,
            'name': 'onion powder',
            'brand': 'McCormick',
            'optional': False,
            'quantity': 1.0,
            'units': 'teaspoon'
        },
        {
            'index': 6,
            'name': 'turmeric',
            'brand': 'Simply Organic',
            'optional': True,
            'quantity': 0.5,
            'units': 'teaspoon'
        },
        {'index': 7, 'name': 'salt', 'brand': 'Morton', 'optional': False, 'quantity': 1.0, 'units': 'teaspoon'},
        {'index': 8, 'name': 'water', 'brand': 'Generic', 'optional': False, 'quantity': 1.5, 'units': 'cups'}
    ],
    'instructions': [
        {'index': 1, 'step': 'Soak the cashews in water for at least 2 hours, then drain.'},
        {'index': 2, 'step': 'Cook the macaroni according to the package instructions, then drain.'},
        {
            'index': 3,
            'step': 'In a blender, combine the soaked cashews, nutritional yeast, garlic powder, onion powder, 
turmeric, salt, and water. Blend until smooth.'
        },
        {'index': 4, 'step': 'Pour the sauce over the cooked macaroni and stir until well coated.'},
        {'index': 5, 'step': 'Serve the vegan mac and cheese hot.'}
    ]
}

In [13]:
print(guard.history.last.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
│   │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
│   │ │ │ user │                                                                                              │ │ │
│   │ │ │      │ Generate a recipe for vegan mac and cheese.                                                  │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ Given below is XML that describes the information to extract from this document and the tags │ │ │
│   │ │ │      │ to extract it into.                                                                          │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ <output>                                                                                     │ │ │
│   │ │ │      │   <list description="What are the ingredients for the recipe?" name="ingredients"            │ │ │
│   │ │ │      │ required="true">                                                                             │ │ │
│   │ │ │      │     <object required="true">                                                                 │ │ │
│   │ │ │      │       <integer format="1-indexed" name="index" required="true"></integer>                    │ │ │
│   │ │ │      │       <string format="is-vegan" name="name" required="true"></string>                        │ │ │
│   │ │ │      │       <string description="Suggested brand for the ingredient (if any)" name="brand"         │ │ │
│   │ │ │      │ required="true"></string>                                                                    │ │ │
│   │ │ │      │       <bool description="Is the ingredient necessary?" name="optional"                       │ │ │
│   │ │ │      │ required="true"></bool>                                                                      │ │ │
│   │ │ │      │       <float format="units-imperial" name="quantity" required="true"></float>                │ │ │
│   │ │ │      │       <string format="units-imperial" name="units" required="true"></string>                 │ │ │
│   │ │ │      │     </object>                                                                                │ │ │
│   │ │ │      │   </list>                                                                                    │ │ │
│   │ │ │      │   <list description="What are the instructions for the recipe?" name="instructions"          │ │ │
│   │ │ │      │ required="true">                                                                             │ │ │
│   │ │ │      │     <object required="true">                                                                 │ │ │
│   │ │ │      │       <integer format="1-indexed" name="index" required="true"></integer>                    │ │ │
│   │ │ │      │       <string name="step" required="true"></string>                                          │ │ │
│   │ │ │      │     </object>                                                                                │ │ │
│   │ │ │      │   </list>                                                                                    │ │ │
│   │ │ │      │ </output>                                                                                    │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ ONLY return a valid JSON object (n